# Import Widget Libraries

In [1]:
%load_ext autoreload   
%autoreload 2
import kyurem
from kyurem.core.idom_loader import reload_bundle
from kyurem.core.widget import WidgetModel

kyurem_client: 0.0.0


In [2]:
from kyurem import Service
service = Service(kh = 'indeed.v0.0.1:612e3403')

In [3]:
import pandas as pd
df = pd.read_csv("indeed.v0.0.1_612e3403_sampled.csv")
data = df.to_dict('records')

In [4]:
service.load_corpus(data, 'parent_title', 'context', 'child_title')

In [5]:
schema = service.get_kh_edge_list()

In [8]:
from kyurem import ExplorerESE


def init(): 
    # Fetch initial view data
    return {
        "subgraph": None,
        "children": service.get_children_node_distributions(),
        "relations": service.get_relation_distribution(),
        "datatable": None,
        "highlight": None,
    }

def focus(node, panel):
    # node can be in one of two formats:
    # node : { "node_label": str, "node_property": str, "node_property_value": str }
    # node : { "node_label": str, "title": str, "uuid": str }
    
    # Since the service functions use the first format, node must first
    # be converted to the proper format
    if "title" in node:
        node = {
            "node_label": node["node_label"],
            "node_property": "title",
            "node_property_value": node["title"]
        }
        
    # TODO: Return nodes in a consistent format from the service
    #       functions to streamline the above
    
    
    # Fetch the neighborhood around the input node 
    result = service.get_node_neighborhood(node)
    datatable = service.get_annotated_corpus(node)
    
    data = { "subgraph": None }
    if panel != "schema":
        data["schema"] = result["schema"]
    if panel != "children":
        data["children"] = service.get_children_node_distributions(node)

    # Use a list comprehension to convert the return format of the relation_dist 
    # to a bar-chart format
    data["relations"] = [
        {"x": relation["label"], "y": relation["count"], "type": type}
        for type, relations in result["relation_dist"].items()
        for relation in relations
    ]
    # TODO: Move conversion code into service
    if datatable["rows"]:
        data["datatable"] = datatable["rows"]
        data["highlight"] = datatable["highlight"]
    else:
        data["datatable"] = None
        data["highlight"] = None
    return data
  
# Create widget 
reload_bundle()
explorer = ExplorerESE({
        "init": init,
        "focus": focus
    }, schema)

explorer.show()

component(7f6e774d88e0, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object at 0x7f6eb06d4670>)

LayoutWidget(Layout(component(7f6e774d88e0, self=<kyurem.widgets.StatefulWidgetBase.StatefulWidgetBase object …